In [12]:
 d0 = "ANTHONY BRUTUS CAESAR CLEOPATRA MERCY WORSER"
 d1 = "ANTHONY BRUTUS CAESAR CALPURNIA"
 d2 = "MERCY WORSER MERCY"
 d3 = "BRUTUS CAESAR MERCY WORSER"
 d4 = "CAESAR MERCY WORSER"
 collection = {"doc0": d0,
         "doc1": d1,
         "doc2": d2,
         "doc3": d3,
         "doc4": d4}
collection

{'doc0': 'ANTHONY BRUTUS CAESAR CLEOPATRA MERCY WORSER',
 'doc1': 'ANTHONY BRUTUS CAESAR CALPURNIA',
 'doc2': 'MERCY WORSER MERCY',
 'doc3': 'BRUTUS CAESAR MERCY WORSER',
 'doc4': 'CAESAR MERCY WORSER'}

In [2]:
boolean_operators = {'AND', 'OR', 'NOT'}

In [22]:
#list of terms : return a list of all terms in collection
def get_terms(Documents):
        terms=[]
        for doc in Documents:
            for term in Documents[doc].split(" ") :
                terms.append(term)
        return terms



In [23]:
#list of unique terms : return a list of unique terms in collection (without repetition)
def get_unique_terms(terms):
    unique_terms = []
    for term in terms:
        if term not in unique_terms:
            unique_terms.append(term)
    return unique_terms


In [24]:
  #document collection terms : return a dictionary {key=docID , value=list of unique terms in doc}
def get_document_collection_terms(Documents):
        docs_colllection={}
        for doc in Documents:
            if doc not in boolean_operators :
                docs_colllection[doc]=get_unique_terms(Documents[doc].split())
        return docs_colllection

In [25]:
  #display dictionary content
def display_dict(dic):
        print("\n")
        for i in dic:
            print (i , " : " ,dic[i])
        print("\n")

In [26]:
#this function takes the collection of documents in a form of dictionary as an input
def term_document_incidence_matrix(collection):
        ## list of terms from the data file collection
        terms=get_terms(collection)
        #list of unique terms
        unique_terms=get_unique_terms(terms)
        #Document collection terms
        docs_terms=get_document_collection_terms(collection)
        #TermDocumentIncidenceMatrix
        term_docs_matrix={}
        for term in unique_terms:
            vector=[]
            for c in docs_terms:

                if term in docs_terms[c]:
                    vector.append(1)
                else :
                    vector.append(0)

            term_docs_matrix[term]=vector
        return term_docs_matrix

In [27]:
#this fucntion takes a term and a terms-document incidence matrix and returns the incidence vector of the term
def term_incidence_vector(term,term_docs_incid_matrix):
    try:
     return term_docs_incid_matrix[term]
    except:
     return "term not found"

In [28]:
#build the term-document incidence matrix
term_docs_incid_matrix=term_document_incidence_matrix(collection)
print("Term-Document incidence Matrix")
#formatted Display
display_dict(term_docs_incid_matrix)

Term-Document incidence Matrix


ANTHONY  :  [1, 1, 0, 0, 0]
BRUTUS  :  [1, 1, 0, 1, 0]
CAESAR  :  [1, 1, 0, 1, 1]
CLEOPATRA  :  [1, 0, 0, 0, 0]
MERCY  :  [1, 0, 1, 1, 1]
WORSER  :  [1, 0, 1, 1, 1]
CALPURNIA  :  [0, 1, 0, 0, 0]




In [29]:
#build the term-document incidence matrix
term_docs_incid_matrix=term_document_incidence_matrix(collection)
print("Term-Document incidence Matrix")
#formatted Display
display_dict(term_docs_incid_matrix)

Term-Document incidence Matrix


ANTHONY  :  [1, 1, 0, 0, 0]
BRUTUS  :  [1, 1, 0, 1, 0]
CAESAR  :  [1, 1, 0, 1, 1]
CLEOPATRA  :  [1, 0, 0, 0, 0]
MERCY  :  [1, 0, 1, 1, 1]
WORSER  :  [1, 0, 1, 1, 1]
CALPURNIA  :  [0, 1, 0, 0, 0]




In [50]:
def query_filteration(query,collection):
    terms = get_terms(collection)
    unique_terms = get_unique_terms(terms)
    qterms=[]
    splitted_query = query.split()
    for qterm in splitted_query:
        if qterm in unique_terms or qterm in boolean_operators:
            qterms.append(qterm)
            
    return qterms
query_filteration("mouhaymen AND WORSER",collection)

['AND', 'WORSER']

In [61]:
def boolean_operator_processing(bool_operator, prevV, nextV):
    if bool_operator == "AND":
        return [a & b for a, b in zip(prevV, nextV)]
    elif bool_operator == "OR":
        return [a or b for a, b in zip(prevV, nextV)]
    elif bool_operator == "NOT":
        return [(not a)*1 for a in prevV]


In [62]:
v1 = term_incidence_vector("WORSER",term_docs_incid_matrix)
v2 = term_incidence_vector("CALPURNIA",term_docs_incid_matrix)


In [63]:
print(boolean_operator_processing("AND",v1,v2))

[0, 0, 0, 0, 0]


In [64]:
print(boolean_operator_processing("OR",v1,v2))

[1, 1, 1, 1, 1]


In [65]:
print(boolean_operator_processing("NOT",v1,v2))

[0, 1, 0, 0, 0]


In [70]:
# Boolean retrieval function
# input : Query
def boolean_retrieval(query,collection):
        #build a terms_documents incidence matrix
        term_docs_incid_matrix=term_document_incidence_matrix(collection)
        bitwiseop=""
        #get the query terms
        qterms=query_filteration(query,collection)
        result=[]
        result_set={}
        has_previous_term=False
        has_not_operation=False
        inc_vec_prev=[]
        inc_vec_next=[]
        for term in qterms :
            if term not in boolean_operators:
                if  has_not_operation:
                    if has_previous_term:
                         inc_vec_next=boolean_operator_processing("NOT",term_docs_incid_matrix[term],inc_vec_next)
                    else :
                        inc_vec_prev=boolean_operator_processing("NOT",term_docs_incid_matrix[term],inc_vec_next)
                        result=inc_vec_prev
                    has_not_operation=False

                elif  has_previous_term:

                    inc_vec_next=term_docs_incid_matrix[term]
                else :

                    inc_vec_prev=term_docs_incid_matrix[term]
                    result= inc_vec_prev
                    has_previous_term=True

            elif term =="NOT":
                 has_not_operation=True

            else :
                bitwiseop=term

            if  len(inc_vec_next)!= 0  :
                result = boolean_operator_processing(bitwiseop,inc_vec_prev,inc_vec_next)
                inc_vec_prev=result
                has_previous_term=True
                inc_vec_next= []

        for i,doc in zip(result,collection):
            result_set[doc]=i
        return result_set

In [71]:
query2= "BRUTUS AND CAESAR AND NOT CALPURNIA"
print("query2 boolean retrieval \n ",boolean_retrieval(query2,collection))

query2 boolean retrieval 
  {'doc0': 1, 'doc1': 0, 'doc2': 0, 'doc3': 1, 'doc4': 0}
